In [50]:
import tensorflow as tf 
import matplotlib.pyplot as plt
import os
from tensorflow import keras
import pandas as pd
from tqdm import tqdm
import glob
import PIL
import numpy as np

AUTOTUNE = tf.data.AUTOTUNE

In [3]:
model = tf.keras.models.load_model('classificatio_model.h5')

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
MobilenetV3small (Functional (None, 1, 1, 1024)        1529968   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 10)                10250     
_________________________________________________________________
batch_normalization (BatchNo (None, 10)                40        
_________________________________________________________________
softmax (Softmax)            (None, 10)                0         
Total params: 1,540,258
Trainable params: 1,528,126
Non-trainable params: 12,132
______________________________________________

In [64]:
RES = 224
N_BATCH = 64

N_CLASS = 6
# batch size
N_BATCH = 64
# epoch 수
N_EPOCH = 50
# learning rate
LR = 0.0001


In [65]:
# 사용안함
# 데이터 셋 호출 
data_dir = "."
train_dir = os.path.join(data_dir, 'seg_train', 'seg_train')
val_dir = os.path.join(data_dir, 'seg_test', 'seg_test')

train_ds = keras.preprocessing.image_dataset_from_directory(
    train_dir,
    shuffle=True,
    image_size=(RES, RES),
    batch_size=N_BATCH
)
val_ds = keras.preprocessing.image_dataset_from_directory(
    val_dir,
    shuffle=False,
    image_size=(RES, RES),
    batch_size=N_BATCH
)

train_ds = train_ds.prefetch(AUTOTUNE)
val_ds = val_ds.prefetch(AUTOTUNE)

Found 14034 files belonging to 6 classes.
Found 3000 files belonging to 6 classes.


In [66]:
def cutmix(images, labels):
    imgs = []; labs = []
    for i in range(N_BATCH):
        APPLY = tf.cast(tf.random.uniform(()) >= 0.5, tf.int32)
        idx = tf.random.uniform((), 0, N_BATCH, tf.int32)

        W = RES
        H = RES
        lam = tf.random.uniform(())
        cut_ratio = tf.math.sqrt(1.-lam)
        cut_w = tf.cast(W * cut_ratio, tf.int32) * APPLY
        cut_h = tf.cast(H * cut_ratio, tf.int32) * APPLY

        cx = tf.random.uniform((), int(W/8), int(7/8*W), tf.int32)
        cy = tf.random.uniform((), int(H/8), int(7/8*H), tf.int32)

        xmin = tf.clip_by_value(cx - cut_w//2, 0, W)
        ymin = tf.clip_by_value(cy - cut_h//2, 0, H)
        xmax = tf.clip_by_value(cx + cut_w//2, 0, W)
        ymax = tf.clip_by_value(cy + cut_h//2, 0, H)

        mid_left = images[i, ymin:ymax, :xmin, :]
        mid_mid = images[idx, ymin:ymax, xmin:xmax, :]
        mid_right = images[i, ymin:ymax, xmax:, :]
        middle = tf.concat([mid_left, mid_mid, mid_right], axis=1)
        top = images[i, :ymin, :, :]
        bottom = images[i, ymax:, :, :]
        new_img = tf.concat([top, middle, bottom], axis=0)
        imgs.append(new_img)

        cut_w_mod = xmax - xmin
        cut_h_mod = ymax - ymin
        alpha = tf.cast((cut_w_mod*cut_h_mod)/(W*H), tf.float32)
        label1 = labels[i]
        label2 = labels[idx]
        new_label = ((1-alpha)*label1 + alpha*label2)
        labs.append(new_label)
    new_imgs = tf.reshape(tf.stack(imgs), [-1, RES, RES, 3])
    new_labs = tf.reshape(tf.stack(labs), [-1, N_CLASS])

    return new_imgs, new_labs

In [71]:
def aug(image, label):
    image = tf.image.random_crop(image, [RES, RES, 3])
    image = tf.image.random_flip_left_right(image)
    return image, label

In [72]:
train_ds = train_ds.unbatch().map(aug, num_parallel_calls=AUTOTUNE).batch(N_BATCH, drop_remainder=True).map(cutmix, num_parallel_calls=AUTOTUNE).prefetch(AUTOTUNE)

TypeError: in user code:

    C:\Users\SEHWAN~1\AppData\Local\Temp/ipykernel_26304/3756215660.py:36 cutmix  *
        new_label = ((1-alpha)*label1 + alpha*label2)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:1383 binary_op_wrapper
        raise e
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:1367 binary_op_wrapper
        return func(x, y, name=name)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:1710 _mul_dispatch
        return multiply(x, y, name=name)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\util\dispatch.py:206 wrapper
        return target(*args, **kwargs)
    C:\Users\Sehwan Yoo\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow\python\ops\math_ops.py:530 multiply
        return gen_math_ops.mul(x, y,

In [63]:
# 사용안함
# train_ds.shape
for image, label in train_ds.take(1): 
    pass

print(label)
# plt.imshow(image[0].numpy().astype('uint8'))
# image[0].shape

tf.Tensor(
[2 4 1 2 5 5 3 2 2 3 2 1 1 2 5 0 3 0 3 3 1 2 3 3 4 2 3 3 2 3 5 5 0 4 4 3 1
 2 3 5 0 4 0 0 3 2 1 2 2 2 0 3 1 3 0 3 5 3 1 3 0 5 3 0], shape=(64,), dtype=int32)


In [6]:
folder_list = os.listdir('seg_train/seg_train')
folder_list

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

In [24]:
# image_list = glob.glob(folder_list + '*.png')
map = pd.DataFrame(columns=['image_path', '0', '1', '2', '3', '4', '5', '6' , '7', '8', '9'])


for folder in folder_list:
    
    images_list = glob.glob('seg_train/seg_train/' + folder + '/*.jpg')
    # print(len(images))
    
    for image_path in images_list:
        # print(type(image))
        # print(images)
        # img = tf.io.decode_image(image_path)
        img = tf.keras.preprocessing.image.load_img(image_path, target_size=(RES, RES))
        # img = tf.image.decode_jpeg(image_path)
        # img = tf.image.resize(img, (RES, RES))
        # img = tf.keras.preprocessing.resize(img)
        # print(img.shape)
        img_tensor = tf.keras.preprocessing.image.img_to_array(img)
        # print(img_tensor.shape)
        # img_tensor = img_tensor[tf.newaxis, ...]
        pred = model.predict(img_tensor[tf.newaxis, ...])
        
        map = map.append({'image_path' : image_path, 
                    '0' : pred[0][0],
                    '1' : pred[0][1], 
                    '2' : pred[0][2], 
                    '3' : pred[0][3], 
                    '4' : pred[0][4], 
                    '5' : pred[0][5], 
                    '6' : pred[0][6], 
                    '7' : pred[0][7], 
                    '8' : pred[0][8], 
                    '9' : pred[0][9]}, ignore_index=True)
        
        
map
        
        



,image_path,0,1,2,3,4,5,6,7,8,9
0,seg_train/seg_train/buildings\0.jpg,9.999683e-01,1.361854e-06,6.906179e-06,1.775450e-05,4.200803e-06,1.492561e-06,8.305454e-21,5.239029e-19,1.054238e-21,9.000374e-19
1,seg_train/seg_train/buildings\10006.jpg,9.997237e-01,7.941216e-06,3.076276e-05,2.037189e-04,1.594497e-05,1.781594e-05,3.137069e-17,9.990489e-16,3.887617e-18,1.921403e-15
2,seg_train/seg_train/buildings\1001.jpg,9.995487e-01,5.398376e-05,6.930529e-05,6.225931e-05,2.628215e-05,2.392603e-04,2.840343e-15,1.261507e-13,7.750182e-16,1.387039e-13
3,seg_train/seg_train/buildings\10014.jpg,9.234645e-04,7.934677e-05,6.307434e-03,9.915576e-01,1.058316e-03,7.389826e-05,2.075920e-10,1.075287e-09,2.879689e-11,1.289411e-10
4,seg_train/seg_train/buildings\10018.jpg,9.999956e-01,5.473261e-07,1.974468e-06,1.080802e-06,5.500685e-07,1.098406e-07,5.761624e-24,1.374467e-21,6.609033e-25,1.484951e-21
...,...,...,...,...,...,...,...,...,...,...,...
14029,seg_train/seg_train/street\9961.jpg,4.800708e-07,1.213376e-05,1.236870e-06,9.785554e-09,1.278704e-07,9.999861e-01,3.701496e-21,1.838479e-21,2.463993e-25,6.507733e-22
14030,seg_train/seg_train/street\9967.jpg,1.079406e-06,4.788884e-05,2.727447e-06,4.970615e-08,7.199447e-08,9.999481e-01,4.822676e-20,7.515236e-20,2.608322e-23,1.128402e-20
14031,seg_train/seg_train/street\9978.jpg,3.327463e-05,4.222116e-04,4.214876e-05,2.131288e-06,6.188242e-06,9.994940e-01,3.029379e-15,1.596870e-15,1.416129e-17,1.507383e-15
14032,seg_train/seg_train/street\9989.jpg,4.436808e-07,4.214704e-06,1.745789e-07,9.935647e-10,1.214095e-08,9.999952e-01,1.840207e-22,4.396410e-22,5.151496e-26,2.647350e-23


In [25]:
map.to_csv('map.csv')

In [8]:
tf.random.uniform((), 0, 64, tf.int32)

<tf.Tensor: shape=(), dtype=int32, numpy=1>

In [13]:
map = pd.read_csv('map.csv')

In [39]:
# print(get_idx('seg_train/seg_train/street\999.jpg'))
# print(get_label('seg_train/seg_train/street\999.jpg'))

map

,Unnamed: 0,image_path,0,1,2,3,4,5,6,7,8,9,idx,label
0,0,seg_train/seg_train/buildings\0.jpg,9.999683e-01,1.361854e-06,6.906179e-06,1.775450e-05,4.200803e-06,1.492561e-06,8.305454e-21,5.239029e-19,1.054238e-21,9.000374e-19,0,buildings
1,1,seg_train/seg_train/buildings\10006.jpg,9.997237e-01,7.941216e-06,3.076276e-05,2.037189e-04,1.594497e-05,1.781594e-05,3.137069e-17,9.990489e-16,3.887617e-18,1.921403e-15,10006,buildings
2,2,seg_train/seg_train/buildings\1001.jpg,9.995487e-01,5.398376e-05,6.930529e-05,6.225931e-05,2.628215e-05,2.392603e-04,2.840343e-15,1.261507e-13,7.750182e-16,1.387039e-13,1001,buildings
3,3,seg_train/seg_train/buildings\10014.jpg,9.234645e-04,7.934677e-05,6.307434e-03,9.915576e-01,1.058316e-03,7.389826e-05,2.075920e-10,1.075287e-09,2.879689e-11,1.289411e-10,10014,buildings
4,4,seg_train/seg_train/buildings\10018.jpg,9.999956e-01,5.473261e-07,1.974468e-06,1.080802e-06,5.500685e-07,1.098406e-07,5.761624e-24,1.374467e-21,6.609033e-25,1.484951e-21,10018,buildings
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14029,14029,seg_train/seg_train/street\9961.jpg,4.800708e-07,1.213376e-05,1.236870e-06,9.785554e-09,1.278704e-07,9.999861e-01,3.701496e-21,1.838479e-21,2.463993e-25,6.507733e-22,9961,street
14030,14030,seg_train/seg_train/street\9967.jpg,1.079406e-06,4.788884e-05,2.727447e-06,4.970615e-08,7.199447e-08,9.999481e-01,4.822676e-20,7.515236e-20,2.608322e-23,1.128402e-20,9967,street
14031,14031,seg_train/seg_train/street\9978.jpg,3.327463e-05,4.222116e-04,4.214876e-05,2.131288e-06,6.188242e-06,9.994940e-01,3.029379e-15,1.596870e-15,1.416129e-17,1.507383e-15,9978,street
14032,14032,seg_train/seg_train/street\9989.jpg,4.436808e-07,4.214704e-06,1.745789e-07,9.935647e-10,1.214095e-08,9.999952e-01,1.840207e-22,4.396410e-22,5.151496e-26,2.647350e-23,9989,street


In [29]:
def get_idx(x):
    # str = x.split('/')
    return x.split('\\')[1][:-4]

In [35]:
def get_label(x):
    return x.split('/')[2].split('\\')[0]

In [37]:
map['idx'] = map.image_path.map(get_idx)
map['label'] = map.image_path.map(get_label)

In [48]:
# get_random_map(1, 2)
np.argsort(map.iloc[1, 2:11].values, axis=0)[-2]
# np.argsort(map.iloc[1, 2:].values, axis=0)
# map.iloc[1, 2:11].values

3

In [49]:
def get_random_map(i, rank):
    choiced = np.argsort(map.iloc[i, 2:11].values, axis=0)[-rank]
    choiced_map = map[map['label'] == choiced].sample(n=1)

    while choiced_map['image_idx'].values == i:
        choiced_map = map[map['label'] == choiced].sample(n=1)
        
    # print(choiced_map['image_idx'])
    # return int(choiced_map['image_idx'].values[0])
    return tf.convert_to_tensor(choiced_map['image_idx'].values[0], dtype=tf.int32)

In [ ]:
def improved_cutmix(image, label): 
    # print(1)
    imgs = []; labs = []

    # for image, label in zip(x_train, y_train):
    for i in range(len(x_train)):
    # for i in range(3):
        # print(label)
        APPLY = tf.cast(tf.random.uniform(()) >= 0.5, tf.int32) 
        # idx = tf.random.uniform((), 0, N_BATCH, tf.int32)
        idx = get_random_map(i, 2) 

        w = RES 
        h = RES 

        lam = tf.random.uniform(())
        cut_ratio = tf.math.sqrt(1.-lam) 
        cut_w = tf.cast(w * cut_ratio, tf.int32) * APPLY
        cut_h = tf.cast(h * cut_ratio, tf.int32) * APPLY

        cx = tf.random.uniform((), int(w/8), int(7/8*w), tf.int32)
        cy = tf.random.uniform((), int(h/8), int(7/8*h), tf.int32)

        xmin = tf.clip_by_value(cx - cut_w//2, 0, w)
        ymin = tf.clip_by_value(cy - cut_h//2, 0, h)
        xmax = tf.clip_by_value(cx + cut_w//2, 0, w)
        ymax = tf.clip_by_value(cy + cut_h//2, 0, h)

        mid_left = image[i, ymin:ymax, :xmin, :]
        mid_mid = image[idx, ymin:ymax, xmin:xmax, :]
        mid_right = image[i, ymin:ymax, xmax:, :]
        middle = tf.concat([mid_left, mid_mid, mid_right], axis=1) 
        top = image[i, :ymin, :, :]
        bottom = image[i, ymax:, :, :]
        
        

        new_imgs = tf.concat([top, middle, bottom], axis=0)
        imgs.append(new_imgs)

        cut_w_mod = xmax - xmin
        cut_h_mod = ymax - ymin

        alpha = tf.cast((cut_w_mod * cut_h_mod) / (w*h), tf.float32)

        label1 = label[i] 
        label2 = label[idx]

        new_label = ((1-alpha) * label1 + alpha * label2) 
        labs.append(new_label)

    new_imgs = tf.reshape(tf.stack(imgs), [-1, RES, RES, 3])
    new_labs = tf.reshape(tf.stack(labs), [-1, 10])
    
    return new_imgs, new_labs